In [118]:
import torch 
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, Seq2SeqTrainer, TrainingArguments, DataCollatorForSeq2Seq, AutoModelForSequenceClassification, Seq2SeqTrainingArguments
import numpy as np
from datasets import load_dataset, Dataset
import pandas as pd
from sys import argv as args
import os
from pathlib import Path
from torch.utils.data import DataLoader, TensorDataset

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [119]:
prefix = "intent classification: "
    
model_name_or_path = '../output/fine-tune_models/SNIPS/model_2/four_shot'
test_set_path = '../data/evaluation/SNIPS/data/test.csv'

dataset = load_dataset("csv", data_files=test_set_path)
dataset = dataset['train']

Found cached dataset csv (/work/pi_adrozdov_umass_edu/vpamidimukka_umass_edu/hf_cache/datasets/csv/default-203601fc4f379794/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

In [120]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [121]:
inputs = [prefix + doc for doc in dataset["text"]]
model_inputs = tokenizer(inputs, max_length=1024, padding = True, return_tensors="pt")
labels = tokenizer(dataset["intent"], max_length=1024, padding = True, return_tensors="pt")

/work/pi_adrozdov_umass_edu/vpamidimukka_umass_edu/envs/vadops/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2372: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [122]:
model_inputs, labels

({'input_ids': tensor([[ 9508, 13774,    10,  ...,     0,     0,     0],
         [ 9508, 13774,    10,  ...,     0,     0,     0],
         [ 9508, 13774,    10,  ...,     0,     0,     0],
         ...,
         [ 9508, 13774,    10,  ...,     0,     0,     0],
         [ 9508, 13774,    10,  ...,     0,     0,     0],
         [ 9508, 13774,    10,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])},
 {'input_ids': tensor([[2334,  304, 2911, 3350,    5,    1],
         [2334,  304, 2911, 3350,    5,    1],
         [2334,  304, 2911, 3350,    5,    1],
         ...,
         [4769, 9937,   53, 8042,    5,    1],
         [4769, 9937,   53, 8042,    5,    1],
         [4769, 9937,   53, 8042,    5,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
         [1, 1

In [123]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model.to(device)
outputs = model.generate(model_inputs['input_ids'].to(device))
# outputs = model.g(input_ids = , labels = labels['input_ids'].to(device))

/work/pi_adrozdov_umass_edu/vpamidimukka_umass_edu/envs/vadops/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [124]:
counter = 0
wrong_examples = []
for i in range(len(outputs)):
    out = tokenizer.decode(outputs[i], skip_special_tokens= True)
    inp = tokenizer.decode(labels['input_ids'][i], skip_special_tokens= True)
    if inp != out:
        wrong_examples.append([dataset["text"][i],inp, out])
        counter += 1
        
print(counter)

120


In [125]:
import pandas as pd
df = pd.DataFrame(wrong_examples)
df.columns = ['text', 'true_label', 'predicted_label']
df.to_csv('../analysis/wrong_examples.csv', index = False)